In [30]:
%%writefile google_ads.py

from __future__ import division
from pyomo.environ import *

# Using the abstact model we will need to specify the index of the model, the variables, the parameters, 
# the equations of the model etc.
model = AbstractModel()

# Definition of the indices
model.i = Param(within=NonNegativeIntegers)
model.j = Param(within=NonNegativeIntegers)

# Definition of the sets
model.N = RangeSet(1,model.i)
model.M = RangeSet(1,model.j)

# Definition of the parameters
model.c = Param(model.N,model.M) # average revenue for company i and query j
model.b = Param(model.N) # budget for company i
model.d = Param(model.M) # estimated requests for query j

# Definition of the variables
model.x = Var(model.N, model.M, domain=NonNegativeReals)
model.z = Var(model.N, domain=NonNegativeReals)
model.y = Var(model.M, domain=NonNegativeReals)

# The objective function
def obj_expression(model):
    return (-1)*summation(model.c,model.x)

model.OBJ = Objective(rule=obj_expression,sense=minimize)

# The budget constraints
def b_constraint(model,i):
    return sum(model.c[i,j]*model.x[i,j] for j in model.M) + model.z[i] == model.b[i]

model.budget_constraint = Constraint(model.N, rule =b_constraint)

# The estimated request contraints
def d_constraint(model,j):
    return sum(model.x[i,j] for i in model.N) + model.y[j] == model.d[j]

model.request_constraint = Constraint(model.M, rule=d_constraint)

Overwriting google_ads.py


In [31]:
%%writefile google_ads.dat

param i := 10; # number of companies
param j := 10; # number of queries

# The average revenue for company i and query j
param c :  1    2    3    4    5    6    7    8    9    10 :=
        1  1    0.75 2.5  2    3    3.5  1.75 0.5  2.5  1.75
        2  0.5  0.5  2    0.75 1.5  1    0.6  0.9  1.5  2 
        3  0.5  2.6  1    2.5  1.5  2.25 0.75 0.5  1.5  0.8 
        4  0.25 0.5  2.8  2    3    3.5  1.75 0.5  2.5  1.75 
        5  0.75 0.5  2.5  1.1  1.4  1.8  1.75 0.5  2.5  3 
        6  1.5  2.75 1.4  2    3.2  3.5  1.9  0.5  2.25 0.9 
        7  1.4  0.7  3.2  2.3  2.1  0.6  1.6  0.3  2.9  3.1 
        8  1    1.3  3.3  3.6  0.65 1.35 1.75 0.85 2    1.95 
        9  0.4  2.1  0.35 2.8  3.45 3    1.25 0.55 2.7  0.2 
        10 2    1.2  3.4  1.65 3.2  2.4  2.8  1.3  0.9  2.4
;

# Maximal budget for company i
param b :=
1 200
2 150
3 180
4 140
5 210
6 190
7 160
8 100
9 205
10 170
;

# Estimated requests for query j
param d :=
1  150
2  90
3  80
4  110
5  135
6  95
7  105
8  120
9  75
10 160
;


Overwriting google_ads.dat


In [32]:
!pyomo solve google_ads.py google_ads.dat --solver=glpk --summary --solver-suffix=.*

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.45] Applying solver
[    0.50] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: -1705.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=100, Index=x_index
        Key      : Lower : Value          : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :            0.0 :  None : False : False : NonNegativeReals
          (1, 2) :     0 :            0.0 :  None : False : False : NonNegativeReals
          (1, 3) :     0 :            0.0 :  None : False : False : NonNegativeReals
          (1, 4) :     0 :            0.0 :  None : False : False : NonNegativeReals
          (1, 5) :     0 :  65.8333333333 :  None : False : False : NonNegativeReals
          (1, 6) :     0 : 0.714285714286 :  None : False : False : NonNegativeReals
        